In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv("data/games.csv")

In [6]:
champ_data = json.load(open("data/champion_info.json"))

In [8]:
df["t1_champ1"] = df["t1_champ1id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t1_champ2"] = df["t1_champ2id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t1_champ3"] = df["t1_champ3id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t1_champ4"] = df["t1_champ4id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t1_champ5"] = df["t1_champ5id"].apply(lambda x: champ_data["data"][str(x)]["name"])

In [10]:
df["t2_champ1"] = df["t2_champ1id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t2_champ2"] = df["t2_champ2id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t2_champ3"] = df["t2_champ3id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t2_champ4"] = df["t2_champ4id"].apply(lambda x: champ_data["data"][str(x)]["name"])
df["t2_champ5"] = df["t2_champ5id"].apply(lambda x: champ_data["data"][str(x)]["name"])

In [11]:
# df only includes champions that are picked
# df = df[["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5", "t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5", "winner"]] 
# df includes champions that are picked and first blood
# df = df[["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5", "t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5", "firstBlood", "winner"]] 
# df includes chapions, first blood, and first dragon
# df = df[["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5", "t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5", "firstBlood", "firstDragon", "winner"]]
# df includes first tower
# df = df[["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5", "t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5", "firstBlood", "firstDragon", "firstTower", "winner"]]
# df includes first baron
df = df[["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5", "t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5", "firstBlood", "firstDragon", "firstTower", "firstBaron", "winner"]]

In [13]:
encodings1 = [pd.get_dummies(df[col], prefix = "t1") for col in ["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5"]]
combined_df1 = sum(encodings1)

encodings2 = [pd.get_dummies(df[col], prefix = "t2") for col in ["t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5"]]
combined_df2 = sum(encodings2)

In [15]:
df = df.join(combined_df1).join(combined_df2)

In [16]:
df = df.drop(["t1_champ1","t1_champ2", "t1_champ3", "t1_champ4", "t1_champ5", "t2_champ1","t2_champ2", "t2_champ3", "t2_champ4", "t2_champ5"], axis = 1)

In [18]:
# import tools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [19]:
# split into features X and labels y
X = df.drop(["winner"], axis = 1) 
y = df["winner"]

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

# initiate and apply the model 
clf = RandomForestClassifier()
clf.fit(X_train, y_train) 

# evaluate the result
clf.score(X_test, y_test)

0.8077296562439309

In [20]:
importances = dict(zip(clf.feature_names_in_, clf.feature_importances_))
sorted_importances = sorted(importances.items(), key = lambda x: x[1], reverse=True)

In [ ]:
# calculate the champion's winrate
champ_name = "Skarner"

wins1 = len(df[(df[f't1_{champ_name}'] == 1) & (df['winner']  == 1)])
wins2 = len(df[(df[f't2_{champ_name}'] == 1) & (df['winner']  == 2)])
losses1 = len(df[(df[f't1_{champ_name}'] == 1) & (df['winner']  == 2)])
losses2 = len(df[(df[f't2_{champ_name}'] == 1) & (df['winner']  == 1)])

print((wins1 + wins2) / (wins1 + wins2 + losses1 + losses2))